## Transfer Learning from InceptionV3

The idea of this step is to leverage existing state of the art models such as Ince

Borrowed some details from this example [https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3](https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3)

# Create image generators

In [23]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image

In [24]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

In [25]:
def save_plots(history,target_file_acc,target_file_loss):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_acc)
    plt.close()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_loss)
    plt.close()

In [26]:
def show_plots(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Doing K-Folds

In [27]:
batch_size = 32
img_height=128
img_width = 256
approx_fold_size = 800

In [28]:
urban_sounds_folder = "/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms"

In [29]:
fold_directories = []
for i in range(1,11):
    directory = urban_sounds_folder+"/fold"+str(i)
    fold_directories.append(directory)

In [30]:
fold_directories

['/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold1',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold2',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold3',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold4',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold5',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold6',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold7',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold8',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold9',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold10']

# MODEL

In [31]:
datagen = ImageDataGenerator(rescale=1./255)

In [32]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from keras import metrics
input_shape = (img_height, img_width,3)
nclass = 10

base_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_height, img_width,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(2000,activation='relu'))
add_model.add(Dropout(0.3))
#add_model.add(Dense(2000,activation='relu'))
add_model.add(Dense(2000,activation='relu'))
#add_model.add(Dense(512,activation='relu'))
add_model.add(Dense(nclass, 
                    activation='softmax'))

#set optimizer
#opt = optimizers.Adam(lr=0.05, beta_1=0.9, beta_2=0.999, decay=0.0)
opt = optimizers.SGD(lr=0.05, momentum=0.9,decay = 0.002)
model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])
#needed to reset weights!
model.save_weights('raw_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2000)              4098000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dense_6 (Dense)              (None, 10)                20010     
Total params: 29,922,794
Trainable params: 8,120,010
Non-trainable params: 21,802,784
________________________________________________________

# Folding

In [ ]:
num_folds = 10 #1 to 10
fold = 0
for directory in fold_directories:
    fold +=1
    #RESET WEIGHTS!!
    model.load_weights('raw_model.h5')
    #
    train_directories = list(set(fold_directories) - set([directory]))
    test_directories = [directory]
    print("Running fold {}, holding data from {} and training on the remaining {}" \
          .format(fold,directory,len(train_directories)))
    train_generator = generate_generator_multiple(generator=datagen,
                                           directories = train_directories,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
    test_generator = generate_generator_multiple(generator=datagen,
                                       directories = test_directories,
                                       batch_size=batch_size,
                                       img_height=img_height,
                                       img_width=img_width)
    history = model.fit_generator(train_generator,
                              steps_per_epoch=approx_fold_size*9/batch_size,
                              epochs=100,
                              validation_data = test_generator,
                              validation_steps=approx_fold_size/batch_size,
                              shuffle=True, 
                              verbose=True)
    model.save_weights('trained_model_fold_{}.h5'.format(fold))
    with open('training_history_fold_{}.json'.format(fold), 'w') as f:
        json.dump(history.history, f)
    save_plots(history,'training_accuracy_plot_fold_{}.png'.format(fold),
               'training_lossaccuracy_plot_fold_{}.png'.format(fold))
    if fold == num_folds:
        break

Running fold 1, holding data from /media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold1 and training on the remaining 9
Epoch 1/100
Found 888 images belonging to 10 classes.Found 873 images belonging to 10 classes.

Found 837 images belonging to 10 classes.
Found 990 images belonging to 10 classes.
Found 936 images belonging to 10 classes.
Found 816 images belonging to 10 classes.
Found 806 images belonging to 10 classes.
Found 925 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Found 838 images belonging to 10 classes.
225/225 [==============================] - 159s 707ms/step - loss: 14.2681 - acc: 0.1111 - val_loss: 14.1033 - val_acc: 0.1250
Epoch 2/100
225/225 [==============================] - 129s 573ms/step - loss: 14.2555 - acc: 0.1156 - val_loss: 14.0437 - val_acc: 0.1287
Epoch 3/100
197/225 [=========================>....] - ETA: 14s - loss: 14.2465 - acc: 0.1161

In [ ]:
show_plots(history)